# 텍스트 마이닝 프로젝트 (2020.6~)
### Data Crawling
#### 휴먼지능정보공학과 류지혜, 이지민

## 에어팟 1세대

In [2]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']

#에어팟 1세대 검색 
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=10776906666&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25201%25EC%2584%25B8%25EB%258C%2580&NaPm=ct%3Dkb3ef8sg%7Cci%3Df7d5699aba286396ee3757965561aa228125c722%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da9cfdf21495248f435f4ae3cbea263c694784ae1"
#xpath
shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[9]/a/span" #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[11]/a/span" #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[10]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[14]/a/span" #휴대성

header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

def save():
    if not os.path.exists('output1.csv'):
        df1.to_csv('output1.csv', encoding='utf-8-sig', mode='w')
    else:
        df1.to_csv('output1.csv',encoding='utf-8-sig', mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            


up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()
            
            
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()
            
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[0]  #에어팟 1세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df1=add_dataframe(name_,category_,reviews,stars,cnt)
save()

페이지 1 

1 2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_< 5 

2 배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:) 5 

3 아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.

할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.

제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.

받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다. 5 

4 다른 저렴한 곳도 있었습니다.

그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.

사용해보니 정말 편합니다. 음질도 나쁘지 않구요.
귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ
적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다 4 

5 택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요
친구들 말만듣고 좋구나...생각만 했는데
막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없

33 하이마트가 혜택이 많아서 구매했는데 배송도 빠르네요. 월 오후 주문이 화 오후에 도착했어요! 집에서 기다렸는데 택배기사님이 연락이나 초인종 없이 문앞에 놓고가셔서 하염없이 기다리긴 했습니다ㅜㅜ
우선 제품은 정말 만족하고 매일 잘 쓰고있어요. 근데 지지직 소리나 음악 채널 변경 시 틱틱 하는 잡음이 좀 있어서 며칠 더 써보다가 서비스센터에 문의해보려해요. 5 

34 음질은 아무래도 유선보다는 약간은 떨어집니다. 하지만 애플 이어팟(유선)과 비교하면 저음부가 풍부해서 어느정도 음질을 보완했다는 생각이 드네요.연결은 매우 빠르고 편리합니다. 애플기기가 2대 이상일때 전환도 매우매우 빠른 점도 좋습니다. 여기에 에어팟을 귀에 착용하면 자동으로 인식하고, 귀에서 빼면 음악이 알아서 정지되는 것도 편하네요.또, 선이 없어져서 운동할때 엄청 편하고 귀에서 잘 안빠집니다. 단순한 음향기기를 구매했다는 관점에서 보면 실망스러울 수도 있는데(비슷한 가격대에 음질 더 좋은 유선 이어폰들이 있으니), 그런걸 떠나서 생활이 꽤 편리해진다는 점이 아주 좋아요. 5 

35 계속 고민하다가 사야겠다 싶어서... 엘페이 등록하고 신한카드 청구할인으로 퀵배송으로 5천원 쓰고 받았습니다. 강남이어서 그런것인지 주문하고 30분 만에 왔어요! 완전 기분 짱이네요 ㅎㅎ 음질 좋고 간편한 연결에 짱입니다. 왜 이렇게 많이들 쓰는지 알겠네요. 5 

36 처음에 출시됐을때는 뭐야? 하는 반응으로 거들떠 보지도 않았는데
아이폰8로 바꾸면서, 이어폰 구멍이 없어져서, 알아보게 됐네요~
가격이 좀 높은편이라 몇일 고민하다가, 그래도 기존 이어폰은
불편할거같아서 구매했는데, 생각보다 귀에서 떨어질 염려도 없는거같고
통화나, 음악 음질도 좋아요!
단점은, 블루투스로 해야된다는점, ^^ 4 

37 이어폰도 분실했고 차량용 블루투스도 불편해서 겸사겸사 비싼 이걸 주문했어요 사용해보니 확실히 편하긴하더라구요 귀에도 밀착감있게 착 붙어서 쉽게 떨어질듯하진 않고. 무엇보다 이어팟을 통해 듣는 소리가 되게 선명

76 에어팟 좋아요~ 평소에는 엘지에서 나온 블루투스 사용하다가 오래 사용해서 전원이 꺼졌다 켜지고 해서 as 하고 그 동안 구매 하고 싶었던 에어팟 구매 했어요. 음질은 저음이 훨씬 풍부 해서 좋았구요. 볼륨이나 다른 기능은 시리 한테 부탁 해야 해서 ㅋㅋ착용감도 가볍고 여름에 좋아요. 한쪽이 삐지면 재생이 중단 되어서 그나마 분실에 대해 조금 덜 걱정 되요. 5 

77 십일절행사때 11일 오후1시쯤? 2주걸린다고되어있을때 주문했는데 17일 금요일 받았습니다~ 와 역시!!! 고민은 배송기간만 늦출뿐이라더니..하 진짜 이 말이 정답이네요... 너무좋아요ㅠㅠㅠㅠ아 진작살걸....정말후회됩니다....!!! 너~무 너무너무너무좋아요!! 착용감도 기존 이어팟보다 조금 더 가벼운느낌? 아무튼 편하고 음질도 아주 쥐똥만큼? 더좋은것같아요ㅋㅋㅋ느껴지네요 정~말 극 미세한 차이랄까 근데 확실히 좋아요ㅋㅋㅋㅋ더 낫습니다 평소에 저처럼 이어팟이 최고였고 이어팟만 쓰던분들은 엄청나게 만족하실거에요! 그리고 제조년월도 최신이에요ㅋㅋㅋ 2017년 11월이네요 좋아요♡ 감사합니다ㅋㅋ 5 

78 오 좋네요. 블루투스 이어폰은 처음인데 선 없는게 정말 편하고 좋네요. 음질도 생각보다 좋아요. 다 좋은데 한가지 단점은 가격이네요. 가격때문에 망설이다가 구입했는데 그래도 잘 한 것 같아요. 배송은 하루만에 바로 왔어요. 오픈형이라 귓속에 집어넣는거 싫으신 분들에게 적합합니다. 4 

79 해외배송으로 오는 다른 곳에서 주문했다가 물건 주문이 무기한 연기가 되서 11번가로 찾았는데 배송도 엄청 빠르고 제품도 2017년 제조 물건이고 안전하게 잘 왔습니다! 돈값을 하는거 같이 편리하기도 하고 음질도 좋네요!! 굿굿굿 5 

80 이어폰 줄때문에 스트레스 받아서 구입한건데 제 폰이 문제인건지...지하철에서 음악듣는데 노이즈가 너무 심합니다 전화할때도 유선이어폰과 달리 마이크가 에어팟 자체에 붙어있어서 듣는사람이 주변소음이 너무 크게들린다고 하네요 3 

페이지 5 

81 배송짱빠름 블투이어폰

207 생각보다 음질이 아주 좋아서 대만족입니다 1 

208 에어팟 가격만큼 합니디.소리도 좋고 5 

209 잘 받았습니다 ! 음질도 좋네요 ㅎㅎ 5 

210 쓰는데 지장없고 음질좋고 좋네요 5 

211 아주 가볍고 안정적인 음질 아주 만족합니다 새로운 세상을 만난 듯하네요 5 

212 음질최고 배송빠름 잘쓸게요 5 

213 소리 휴대성 모두 좋습니다. 5 

214 편하네요 음질도 좋아요 4 

215 동생 생일선물 주려고 샀는데 하루만에 와서 놀랐어요!
음질도 좋고 선 없어서 편하대요👍🏻👍🏻 5 

216 음질 짱 5 

217 음질도 좋고 마음에 들어요~! 마니 파세요~! 5 

218 비싸긴 하지만..좋아요..소리가 약한듯.. 5 

219 역시작품이라 음질도좋고 사용편하네요 5 

220 음질도 품질도 좋습니다 5 

페이지 12 

221 음질 선명하고 커넥팅이 잘되요 4 

222 배송이 하루만에와서 굳선없이 노래 들으니까 어색ㅎㅏ지만 짱편해요!한쪽씩 터치 설정도되고,음질도 나쁘지않구요빠른배송 감사합니당~ 4 

223 배송 완전 빠르네요. 조카 졸업선물로 해줬는데 좋아하네요. 정품 맞구요. 음질도 좋다해요. 이래서 에어팟 하나봐요 5 

224 귀가 커서 조금 안맞긴 하지만 기능상으로나 너무 마음에 들어요 음질도 좋고 그리고 되게 편리해요 5 

225 음질도 좋고 통화음질도 좋네요.굿입니다.너무 맘에들어요. 적극추천합니다. 5 

226 너무 편하고 음질도 좋고 진작 구매할껄 그랬어요.
6만원짜리 이어폰 3일만에 분실ㅠㅠ 4 

227 롯데택배로 바로 보내니 배송도 빠르고 배송조회도 잘되고요 ㅎㅎ 신한카드에 엘포인트 등등 혜택도 많네요! 에어팟으로 삶의 질 올라가네요. 선없는 세상 최고에요! 쿵쿵 음질도 좋고요!! 아이폰이랑 페어링 잘되고 최고에요!! 5 

228 너무 간편하고 음질도 좋아요 최고입니다
고민끝에 비싼값을 지불하기는 했지만 막상 받고나니 만족도는 두배입니다 아깝지않아요 5 

229 배송 빨랐어요.에어팟 진짜 편하고 

344 비오는날 우산이랑 줄이랑 엉켜서 난리나는경우없어서 너무좋고 운동하거나 뛰댕길때도 선이없어서 너무 편리해요 음질도 나쁘지않고 키링달면 귀엽고좋습니다 고민중이시라면 꼭 사세요 ㅠㅠ존좋 5 

345 음질도 생각보다 좋고 무엇보다 편리합니다. 추천 제조일자도 18년3월이네요 굿굿 5 

346 음질은 이어팟이보다 조금 더 좋은거 같아요 편리하고 너무 좋아요 5 

347 첨에 그냥 저렴한 일반 블투이어폰을 썼다가 너무 쉽게 고장나서 큰 맘 먹고 정품샀는데 너무너무 좋아요!!! 부드럽고 음질도 괜찮아요.. 크기도 부담스럽지 않고 배송도 엄청 빨라요. 추천합니다 5 

348 배송좋음! 잘 사용하고있음! 줄이없으니 일단 너무 편하네요 운동할때 너무 불편했어서 주문한건데 생각보다 배송도 빠르고 음질도 나쁘지 않네요! 4 

349 음질도 너무 좋고 ㅠㅠㅠㅠ 배송도 발리 받았네요~ ㅠㅠㅠ 좋은 상품 감사합니다~ 5 

350 일요일 새벽에 주문해서 수요일 오후에 받았습니당 배송은 빨랐고 에어팟 열자마자 페어링 속도도 빨라요전 하자 없고 음질도 진짜 좋아요 ㅋㅋㅋㅋ후기보며 걱정 많았는데 좋아용 5 

351 배송도 하루만에 빨리 오고 너무 좋아요!
음질도 좋습니다! 살까말까 할때는 역시 사는게,, 5 

352 음질도 생각보다 좋구 디자인도 귀엽고 좋은데자석부분에 검은색 철이 계속붙네여ㅜ 5 

353 역시 너무 편리해요.
음질도 소름끼치게 좋고.
포장도 튼튼하게 잘 왔어용!! 5 

354 배송도 음질도 최고! 선물해야하는거라 온라인에서 주문해도 되나 걱정하면서 주문했는데 배송도 빠르고 음질도 상당히 좋네요! 4 

355 에어팟 음질도 좋고 배송도 빠릅니다 사업 번창하세요! 5 

356 남자친구선물로 샀어요 ㅎㅎ
음질도 좋고 무엇보다 배송이 넘 빠르네요
감사합니다 4 

357 역시 다르다는 느낌.
소리가 우선 맘에 들고,
무선의 편리함과 siri의 편리함을 만끽합니다.
가격의 값어치 한다는... 5 

358 왜 에어팟에어팟 하는지 알겠어요 솔직히 너무좋아요


...

15 5프로 쿠폰 발급과 롯데 포인트 사용해서 나름적당한 가격에 구매한 것 같습니다. 배송은 주문한 날 바로 시작되서 빨리 받을 수 있었습니다. 역시 써보니 선이 없어 불편하지도 않고 이어팟보다 귀에 더 잘맞고 좋네요. 페어링도 빠른편이고 기계치인 저도 쉽게 쓸 수 있어 좋습니다. 감사합니다. 4 

16 일단 빠른배송은 인정합니다!!비록 물품을 받고 사용시간은 얼마 안되지만 너무도 편리하네요..일단 무선이란게 너무 큰 메리트입니다.. 페어링도 빠르고..너무 간편합니다!!다만...단차가 좀 있는게 아쉽네요 이런거 예민한분들은 고민하시고 구매하셔야할꺼같아요 물론 단차라는게 큰건아니고..이어팟??과 비슷합니다 여튼 전 단차 상관없이 너무 편리해서 맘에드네요 굿굿 4 

17 마침 각종 쿠폰 사용하여다른 곳보다 저렴하게 잘 구입했습니다.그래도 비싸긴 하지만요.기존에 사용했었는데 너무 좋아서선물하려고 재구입했습니다.보스 이어폰보다 성능이 훨씬 뛰어나네요.배송도 빠르구요.잘 사용하겠습니다. 5 

18 아이폰 사용자여서 이어폰잭이 없어 충전과 이어팟을 같이 못 써서 매우 불편해 큰 맘 먹고 샀네요. 이어팟 줄 풀 때마다 가위로 자르고 싶었는데 에어팟은 신세계네요 ㅎㅎㅎ 다만 성능 대비 가성비가 너무 떨어지고 음질도 그닥 좋지도 않네요… 편의성은 단연 모든 블루투스 이어폰 중 최고입니다. 돈이 넘치는 분에게 추천합니다. 3 

19 성능이 아주 좋습니다. 5 

20 성능이좋대요배송도빨라요ㅎ 5 

페이지 2 

21 성능 짱 좋습니다. 5 

22 배송 정말 빨라요..! 제품도 잘 받았고요!질문답변에서 보고 정품인거 믿고 주문했습니다. 블루투스 연결도 빠르고 첫 무선 이어폰 구매인데 조심히 잘 사용해 볼게요. 다른곳보다 더 저렴하게 살수있어서 너무 좋았어요! 5 

23 최저가로 구매해서 좋아요! 음질 성능 단연 최고입니다^^ 5 

24 디자인 심플, 성능 굿굿.. 5 

25 배송도 빠르고 성능도 너무 좋고 이쁩니다😍😍 5 

26 성능이 너무 좋아요 기대이상입니다

7 제품 크기에 비해 큰 박스에 넉넉한 뽁뽁이에 넣어 주셔서 안전 배송을 해주신거 감사드려요.
정품이니 상품은 문제 없는거 같구요, 정말 편하고 페어링 잘 됩니다. 떨어지진 않는데 처음이라 신경이 많이 쓰이네요. 잘 쓸거 같아요 4 

8 아주 빠르게 배송받았습니다.아이폰X 쓰는데 에어팟을 열기만했는데 자동으로 페어링이 되네요본체배터리 좌우 에어팟 각각 배터리 잔량이 정말 예쁘게 뜨는데 이래서 에어팟 에어팟하나 싶네요1년전에 사지않은걸 후회합니다. 5 

9 아주 빠르게 배송받았습니다.아이폰X 쓰는데 에어팟을 열기만했는데 자동으로 페어링이 되네요본체배터리 좌우 에어팟 각각 배터리 잔량이 정말 예쁘게 뜨는데 이래서 에어팟 에어팟하나 싶네요1년전에 사지않은걸 후회합니다. 5 

10 쇼킹딜에 뜨는 싼 에어팟 사지 마시고 여기서 사세요!!!!! 거기서 구매한 리뷰자분들의 페어링하면 이미 ㅇㅇ의 Airpods 같이 다른사람 이름으로 페어링된대서 가격이 조금 있어도 믿고 구매했어요. 저는 페어링하니까 바로 제 아이폰 이름으로 제이름의 에어팟 이렇게 떴습니다!! 빨리 받고싶어 퀵으로 거래했고 너무 친절하게 응대해주셔서 주문한지 4시간도 안되어 받았습니다!! 작동 잘 되고 제조일자도 확인되고 뽁뽁이도 정말 안전하게 해주시고 감사합니당!! 단골해놓고 다음에 또 구매할 의사 있어요!!! 리뷰 진짜 안쓰는데 사장님 너무 좋으셔서 ㅠㅠㅠ 쓰고갑니다!!! 번창하세요! 5 

11 하이마트 특가이벤트로 장만했는데 하루만에 배송받아서 바로 사용할 수 있어서 정말 좋습니다 기존 이어폰 줄때문에 너무 번거로워서 별 기대없이 샀는데 사운드도 좋구요 케이스도 그냥 케이스가 아니더라구요 ㅎㅎ 착용하면 바로 자동으로 인식되는 기능도 좋아요^^ 5 

12 음질 : 맥 프레 2017년 형 페어링 시에 이상하게 주변에 이전 연결한 아이폰이 있으면 가끔 끊기면서 들릴 때가 있습니다. 이유는 명확히 모르겠네요; 배송 : 보통입니다. 평가 : 우선은 맥이나 패드 아이폰 같이 쓰는 분들은 멀티 페어링이 안

3 진짜 빠른 배송이였어요 ! ! ! ! !
주문하고 이틀만에 받았어요. 정말 빠르네요 !!
직구를 할까 알아봤지만... 가격도 얼마 차이안나고 그냥 쓱 닷컴 믿고 구매했는데 에어팟 제조일도 한달밖에 안됐고 배송 빨라서 마음에 쓱 들어요.
노트북으로 영화볼 때도 블루투스 연결해서 사용하니까 정말 편하네요. 다만 영화 3편 보니까 배터리 충전해야된다는..
다들 왜 진작 구매하지않을까 라는 평에 혹 해서 얼른사자 싶어서 샀는데 백번 공감해요!
고민하지말고 얼른 살걸그랬어요.
근데 귓구멍 작은 분들은 좀 아플거같아요.
전 오래사용하면 좀 아프네요 ~ 그래도 다른 이어폰도 다 아프고 불편하니까요...
그래서 너무 꽉 끼게 넣어서 그런가 싶어서 꽉 눌러 넣지않아도 진짜 귀에서 잘 안떨어져요.
가방에 비와서 우산까지 들어야 할 때 진짜 편할 거 같아요.
사은품으로 충전선을 주셔서 에어팟 기본으로 들어있는 아이폰 8핀 충전선까지 !!! 충전선이 2개나 생겼어요 !!
빨리 배송 받고싶은 분 들은 여기서 빨리 구매하세요. 5 

4 광고로만 보던제품을 직접사용해보니 넘 좋더라구요.간편한건 당연~~~음악을 들으니 콘서트에 온것같아요.화장하면서도 줄이 꼬이지않아 편리하구 잠잘때 asmr을 즐겨듣는데 실감나요.다만 크기가작으니 분실위험이 있어 불안해요.욕실도 가지고 들어가면 안될거같구...길가면서 음량을 너무 크게 켜면 자동차소리가 잘안들리니 주의해야할것같아요. 5 

5 사실 다른 코드프리 이어폰들과 비교해서 비싼 가격이라고는 생각되지 않습니다.
다른 제품들을 많이 사용했지만 에어팟만큼 완성도가 높고 편리한 제품은 많지 않다고 생각합니다.
다른 애플기기들과 사용할 때에는 그 어떤 이어폰들이 상대가 되지 않을 정도로 편리합니다.
음질도 블루투스로서는 만족할만한 수준이고, 배터리도 오래가고, 충전할수 있는 케이스에 탈착하는 것도 다른 이어폰들보다 정말 편하고 깔끔합니다. 5 

6 왜 무선이 답인가!!! 에어팟 첫 이미지는 너무 작고 분실의 위험이 떠오르는데요..막상 사용시 귀에 쏘

3 갑자기 에어팟에 꽂혀가지고 여기저기 알아보다가 하이마트에서 파는걸 발견했는데 청구할인이랑 l포인트도 적립되고해서 나쁘지않은 금액으로 잘 산 것 같아요!ㅎㅎㅎ
배송도 빨라서 좋았구요 꼼꼼하게 잘 포장해서 주셔서 감사해요~! 에어팟은 성능도 좋고 잘 작동하구요 그냥 보기만해도 너무 귀엽고 예쁘네요ㅎㅎㅎ계속 끼고있고 싶어요~~ 착용샷도 괜찮고! 진짜 비싸서 고민 많이했는데 그만큼 만족감도 커서ㅋㅋㅋㅋ주변사람들한테도 추천중이에요!!ㅋㅋㅋ 5 

4 많은 리뷰에서 높게 평가했듯이 동일 iCloud 계정을 공유한 디바이스들 사이의 연결성이 꽤 좋습니다. 디바이스와 의 첫 연결은 매우 심플하고 어찌보면 꽤 세련된 방식을 사용하고 있습니다. 충전기의 덮개를 열고 연결하고자하는 디바이스 근처로 가져가면 디바이스에서 바로 AirPod을 인식, 연결을 할 수 있으며, 초기 연결이 완료되면 앞서 이야기한 것처럼 동일 iCloud 계정을 공유하는 디바이스들(iPad, iPhone, Apple Watch, iPod Touch, Mac)의 AirPlay 항목에 자동으로 AirPod이 등록되어 쉽게 연결 디바이스를 옮겨 사용할 수 있습니다. 음질은 양호한 편이며 비슷한 시기에 출시된 비슷한 가격대의 PowerBeats 3 Wireless Earphones (애플의 자회사인 Beats 제품)보다 음질은 깔끔한 편이며, 애플에서 제작한 Apple In-ear Headphone처럼 . 일반 이어폰은 귀에 잘 걸리지 않아서 망설인 부분이 있었는데, 생각보다 귀에 잘 걸려 있으며, 인이어가 아님에도 베이스를 잘 유지해줍니다. 양 이어셋에는 버튼이 없는 깔끔한 디자인이며, 대신 상단 표면을 더블탭 해서 시리를 부르거나 음악 플레이 조작을 할 수 있습니다. 각 이어셋마다 다른 기능을 설정, 최대 2개의 기능을 설정해서 사용할 수 있습니다. 사용이 편리하지만 물리적인 볼륨 조작이 불가능하며, 오직 시리에게 명령해서 ('Turn volume to 25%', 'Turn up the volume' 등) 

108 너무 예쁘고 좋아요!!! 5 

109 신속히 도착 되었습니다. 예상보다 상품이 속히 도착되었습니다. 배송준비나 처리가 빠르게 진행이 되었습니다. 제품의 품질은 물론이고 성능과 디자인이 다른 것보다 우수하여 만족스럽습니다. 4 

110 실물은 훨씬 더 작고 이쁘네요 성능도 마음에 들고 배송도 빨랐습니다 감사합니다 5 

111 실제 사용해보니 음질도 괜찮고 편리하네요.
디자인도 괜찮습니다. 구매추천합니다. 5 

112 음질, 디자인, 착용감 최고 입니다. 단지 분실에 대비해서 뭘 또 준비해야 할것 같아요^^ 배송이 조금만 늦었으면 다른거 사려고 했는데, 총알배송이네요~ ㅋ 5 

113 상품 발송준지와 처리가 신속하게 이루어져 만족스럽고 제품의 디자인과 품질이 좋습니다. 3 

114 디자인도 이쁘고 소리도 좋고 만족합니다! 5 

115 깔끔하니 좋아요 감사합니다 잘쓸께요왜 에어팟에어팟 하는지 잠깐 ？㎢쨉？ 확느껴지네요 5 

116 타 사이트보다 저렴하게 배송도 빠르고 디자인도 이쁘고 좋아요.잘 쓸게요 4 

117 쇼킹딜때 저렴하게 구입해 2주정도 기다려야된다고하여 마음비우고 있었는데 12월16일 수령했어요.음질.디자인 다좋네요.너무만족스럽네요 5 

118 맘에 듭니다 디자인, 성능 모두 만족합니다 5 

119 이쁘고좋아요저쓰다가와이프가 고민해서 하나 더 샂 ㅜㅆ네요 운동항때 특히 좋아요 아무리 뛰어도 빠지지 아늠 5 

120 생각보다 작고 귀엽고 좋네요~잘사용 하겠습니다 4 

페이지 7 

121 굿 굿스피드배송 굿굿 품질명도 좋고 스타일도 좋습니다나중에 또 구입하겠습니다 굿굿 감사합니다 5 

122 깔끔하고 좋네요^^ 5 

123 이틀만에 왔습니다 ㅎㅎ 포장은 에어캡 있는 포장지로 잘 되어 왔네요. 실물 보고 연결하자마자 너무 좋네요!! 5 

124 배송 빨랐고 제품도 깔끔하고 좋습니다. 5 

125 역시 모양도 이쁘고 무선이라 좋아요. 2번째 구입해요^^ 가족중에 아이폰 사용하는 가족이 많아서 한개는 가족한테 넘기고 한개 또

## 에어팟 2세대

### 에어팟 1세대와 2세대의 가장 큰 차이는 W1->H1칩 발전

H1은 동기화가 개선됐다. 즉, 에어팟으로 전화를 받을 때 훨씬 빨리 연결되고, 다른 디바이스로의 연결도 더 빨리 진행된다. 애플에 따르면, 에어팟은 디바이스간 연결 전환이 최대 2배 빨라졌고, 통화 연결 시간이 최대 1.5배 빨라졌다고 한다.(품질,성능)

통화 가능 시간도 더 길어졌다. H1의 배터리 효율이 높아졌기 때문이다. 1세대 에어팟은 통화 가능 시간이 2시간이었는데 H1 칩을 탑재한 2세대 에어팟은 총 3시간이다. 음악 감상에는 최대 5시간을 사용할 수 있으며, 케이스를 활용해 총 24시간을 사용할 수 있다. 15분 충전으로 3시간의 음악 감상 및 2시간의 통화가 가능한 수준으로 충전할 수 있다.(배터리 수명)

H1은 현재 귀에 착용하고 있는 에어팟의 수(한쪽인지, 두쪽인지, 아예 없는지)를 광학 센서와 모션 가속도계에서 받은 정보로 파악한다. 그 다음 여기에 맞춰서 오디오를 전송하고 통화할 때 외부 노이즈 감소 시스템을 조정한다.(음질,기능,품질,성능)

더불어 1세대 에어팟의 문제점, 특히 게임을 할 때 문제점 중 하나로 꼽혔던 레이턴시(latency=반응속도)가 감소했다. 애플에 따르면 H1 칩으로 레이턴시가 최대 30% 감소했다.(성능)

음성으로 시리를 호출할 수 있게 된 것도 큰 개선점이다. 이제 아이폰을 들지 않아도 “시리야”라고 말함으로써 음악 재생을 제어하거나, 전화를 걸거나, 메시지를 보내거나, 방향을 알 수 있다.(조작성, 사용성,기능)

음질, 디자인,착용감,휴대성 차이 없음.

만족도(카테고리 있음)비교도 좋을듯

원문보기:
http://www.itworld.co.kr/news/119204#csidxd40396c08da7bd6bcac69035a026bc9 

In [6]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']

#에어팟 2세대 검색 (유선)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=18622086330&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25202%25EC%2584%25B8%25EB%258C%2580%2520%25EC%259C%25A0%25EC%2584%25A0%25EC%25B6%25A9%25EC%25A0%2584&NaPm=ct%3Dkbb1kefs%7Cci%3D145113ad84d3bb4c745b6faa37d4d9894a9d33fb%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dad9562496841c80a6f639b5837f07d5e88b01927"

shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[4]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[9]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[15]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[12]/a/span"  #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[13]/a/span"  #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[11]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span" #휴대성



header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df2=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df2.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df2.loc[n]=[name,category,'null','null']
        n+=1    
    return df2

def save():
    if not os.path.exists('output3.csv'):
        df2.to_csv('output3.csv', mode='w')
    else:
        df2.to_csv('output3.csv',mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

페이지 1 

1 정말 마음에 들어요! 중고로 사려다 아무래도 불안해서 믿을 만한 곳 찾아보다 여기서 골랐는데 안전하게 사길 잘했네요. 제조년월도 2020년 2월로 최근이구요. 깔끔합니다.

정품인증도 숫자0을 영문O로 잘못봐서 안된다는 메시지에 식겁했는데 제 실수였습니다ㅎㅎ 구매한 날짜는 어떻게 등록하는건지 몰라 아직 등록 전이네요ㅠ

놀랐어요. 블루투스 켜고 뚜껑 여니까 바로 화면에 뜨네요. 화면 캡처한건 첨부가 안돼서 맥북에 연결하면서 찍은 걸로 올립니당. 볼륨 조절할 때 뜨는 콩나물 ㅋㅋ 5 

2 @가격 : 에어팟 2 요즘에 그냥 다 이 가격대 인것 같아 싸게 산것 같진 않고 그냥 보통? 그냥 그 가격; 이미 프로도 나왔으니...
@기기 : 뭐 요즘 짭 많다 하는데 내가 짭을 경험 해본 경우가 없어서... 근데 수령하고 에어팟 열면 아이폰 같은 경우 자동 등록 되는것 같고 뭐 상세한 설정 같은것; 아이폰 사이트 보증 등록, 본인 기기 설정, 등 만 조금만 하면 되고 기기 자체는 상태 괜찮은 듯 하다. 나는 조금 늦은 3월에 샀는데 20년 01월 제품으로 들어옴.
@에어팟 프로와 비교 : 물론 개인 만족 이지만 나는 실제 프로와 2 둘다 봤는데 솔직히 프로가 더 작고 디자인도 더 편해보이고 작아서 디자인 면에서는 프로가 괜찮지만 나는 개인적으로 커널형 이라지만 앞에 고무 달린 이어폰을 개인적으로 별로 안좋아 해서 ; 이물질 같은것도 더 많이 묻고 닦다 보면은 고무도 늘어나고 에어팟 기존의 형태가 . ...
@무선과 유선 : 요즘에 뭐 무선 충전 많이 하고 카페 같은데 가도 무선 충전기 들이 다 배치 되어 있어서 편하긴 하지만. 솔직히 프로 사는거 아니고 나와 같이 에어팟2 사는데 무선 유선 고민 한다면 뭐 갠취 지만 나는 유선 추천. 어차피 이 허얗고 조그만 한거 그 카페에서 무선 충전 한다고 냅둬 봤자 언젠가 잃어 버린다. 어차피 집에서 충전을 가장 많이 할텐데 둘중에 사야 한다면 그냥 유선 사는게 그나마 가성비다. 5 

3 제품 매우 만족. 1세대

33 우선 사길 잘했다 생각이 듭니다. 무선 이어폰이 출시되고 나서 구매하고 싶었는데 약 20만원에 가격을 이어폰에 못 쓰겠더라구요. 그래서 3만원대에 가성비 좋다는 중국제품 무선 이어폰을 사서 사용하고 있었습니다. 하지만 주변 지인이 사용하는 에어팟 1세대를 이용해보니 좋아서 사고 싶은 생각이 다시 들더라구요. 무엇보다 귀에서 안 빠져서 너무 좋습니다. 다른 타 제품은 귀 입구쪽이 실리콘재질이여서 사용하다가 빠져서 불편했었어요. 콩나물 디자인 짱!! 착하지 않은 가격빼고 모두 만족합니다. 5 

34 일단 선이 없다는게 제일좋네요. 어차피 선 없는 이어폰이 필요했구 에어팟외형이 맘에 들어 구입했으니 후회는 없습니다~음질은 유선이어폰과 흡사해요. 통화시 주변소음 상대에게 들린다고하네요. 모바일게임 중 가끔 블투연결이 끊키곤 해서 주변에 겜소리 들리게하여 민망하네요.배터리가 생각만큼 오래가지 않아 자주충전 해주고 있어요. 5 

35 한달하고 몇일 지나서 받았어요사전예약이라 다른 판매자분들 보다더 많이 스트레스 받으셨지 싶어요저도 통신 판매하는 사람이라그런 진상은 부리고 싶지 않았네요ㅋㅋ..주말에 추가금 입금하고 오늘 바로 받았어요금액이야 어찌 됐던 구매했음에 만족합니다 5 

36 하이마트에서 상품 보고 와서 H 몰 에서 구매를 하게 되었습니다 그런데 H 몰 이 훨씬 싸고 좋았습니다.가격 차이가 10% 이상 세요 왔습니다.말은 배송해 상품을 받았는데하고 똑같습니다 지금 현재 아주 잘되고 있습니다 강력 추천 한다 5 

37 후기에 100% 정품이라는 말에 바로 구매했구요
오프라인에는 전부 품절이라 알아보다가 구매했는데요ㅠㅠ
정말 저렴하게 좋은 제품 잘 구입했습니다!!
살짝 끊김이 느껴져서 긴장했는데 블루투스는 어느정도 그럴 수 있다고 해서 이해하기로 했어요
그래도 너무 좋아요 거의 하루종일 이어폰끼고 일하신다면 강력추천이요!!! 5 

38 배송도 빠르게 왔고
상품도 괜찮네요 ㅎㅎㅎ
11**에서 1만원 저렴한 것도 다른 판매자가 팔고 있던데 후기가 안 좋아서 

페이지 7 

121 하나사고 선물용으로 구매했는데 만족합니다. 5 

122 미개봉 정품 왔습니다 ~ 사길 잘한것 같아요 5 

123 배송이 엄청 빠르네요 ㅎㅎ 상품도 좋아요!! 4 

124 빠른배송 감사합니다 제품도 아주 맘에듭니다~~^^ 5 

125 빠른배송 좋은제품 만족합니다! 5 

126 저렴하게 좋은상품 감사합니다 5 

127 고민했는데 사길 잘한거같아요 5 

128 배송.상품 모두만족합니다 5 

129 내가 이게 과연 필요한 걸까 하면서 반신반의하고 사보았어요. 신랑것도 같이 샀는데 와...진짜 신세계...왜 진작 사지않았을까하는 후회뿐.
다른 후회는 전혀 없어요.
케이스 바꿔끼우는게 아주 재미남 5 

130 가격대비괜찮은제품이네여작동잘되고음질도좋네여~~~~ 5 

131 에어팟을 떠나보내구 다시 재구매 엉엉
품질, 가격대비 짱이랍니다 이젠 내곁에 꼭함께해야해 5 

132 역시 에어팟 좋아요. 사길 잘한거 같아요 5 

133 좋은 제품을 저렴한 가격으로 잘 샀네요. VVIP고객 되니 할인율이 높아 기분 좋네요^^ 딸아이 어린이날 선물로 줬더니 행복해하네요~ 5 

134 구매하길 정말 잘한것 같아요. 직접 사용해보니 만족도가 높습니다. 4 

135 두번째 구매이고 만족스러워요.저렴하고 배송도 빨라요. 5 

136 배송 빠르고 물건도 만족합니다.
애플 코리아 정품 등록 잘 되었구요
통화 음질도 괜찮습니다. 5 

137 좋은상품 저렴하게 잘 샀습니다.아이가 좋아하네요. 5 

138 배송은 보통이었고 제품은 무척 마음에 듭니다. 4 

139 가격도 좋고 제품도 만족스러웠어요 5 

140 배송도 빠르고 상품도 만족합니다 5 

페이지 8 

141 배송도빠르고 물건도 좋았습니다 5 

142 배성빠르고 물건도 좋습니다 5 

143 빠른배송! 물건도 좋아요! 5 

144 저렴한 가격에 좋은 상품 가성비 아주 좋습니다 !! 근데 배송이 많이 느렸어요 ㅠㅜ 그래도 상품 상태 아주좋고 그래서 만족합니다 !! 4 

145 제품 만

298 배송빠르고 상품도 만족합니다 5 

299 가격대비아주좋고 정품입니다 5 

300 너무 만족하고 쓰고 있습니다 5 

페이지 16 

301 배송이 빠르고 제품도 만족스럽습니당프로랑 고민했는데,, 만족해요^^ 5 

302 빠른 배송..상품도 만족 5 

303 포장 꼼꼼하게 잘 해주셨고 상품도 너무 좋아요! 배송이 좀 느렸는데 아마 설날 연휴 기간이어서 그런 것 같습니다! 배송 빼곤 다 만족해요:) 4 

304 상품 만족합니다..... 4 

305 생각보다 배송이 빨리 와서 좋았습니다진작 살걸 왜이제야 샀나 싶어요할인권 이용해서 싸게 구입해서 더 좋아요좋은상품 감사합니다^^ 5 

306 가성비 굿..싼 가격에 적당하게 사용 가능한 제픔이네요..근디 오른쪽 구성품이 자주 끊겨요.. ㅎㅎ;; 4 

307 배송 빠르고 제품도 만족합니다. 감사합니다. 5 

308 자잘한 오작동마저 없이 아주 만족하며 잘 사용하고 있습니다 정품 저렴하게 사서 더 좋구요 5 

309 빠른배송 감사합니다
제품 만족합니다 5 

310 배송이 너무 너무 빨라요정품 등록 너무 너무 잘되어요강력 추천해요 5 

311 멍절이 끼여 다소 배송이 지연됐지만 제품은 만족합니다 5 

312 배송겁나빠르고 상품도완전좋아요
에어팟은 사랑입니다 5 

313 새상품에다 흠집하나 없는 상품이라 너무 좋습니다. 케이스가 소진되서 못받을줄 알았는데 받았네요. 감사합니다^.^
남편이 에어팟 노래를 불러서 사줬는데
막상 부끄럽다고 밖에 못하고 다니네요
ㅋㅋ 5 

314 아들 선물해줬는데 만족하니 잘 쓰고 있어요 4 

315 고민고민하다가 구매 했습니다.
갤럭시 버즈랑 에어팟 사이에 고민 많이 했는데..
저는 갤럭시를 쓰지만 에어팟으로 구매 했어요..
만족하면서 사용하고 있습니다.[G마켓 안드로이드 앱에서 작성] 5 

316 상품만족합니다. 또 구매하고 싶네요. 4 

317 아주 만족합니다실리콘 케이스도 같이 와서 더더욱 만족스러워요심지어 2020년 제작 제품 이라서 너무 만족해요 5 

479 배송과 상품 모두 만족합니다. 가격도요. 5 

480 배송도 빠르고 상품도 맘에 들어요~ 5 

페이지 25 

481 상품 만족합니다. 안전포장으로 상태도 좋아요.
다만, 분리배송이 필요해서 배송비 3번 내고 주문했는데, 한번에 합배송으로 왔네요. 판매자님 배송비 아끼셔서 좋겠지만, 해당부분은 불만입니다. 5 

482 좋은 상품 저렴하게 구매했네요 5 

483 저가에 사서 가성비 좋아요. 4 

484 그냥 그렇고 가격값 합니다 1 

485 배송빠르고제품만족합니다 4 

486 기다리다가 구입했는데 매우 만족합니다배송도 빨랐구요 완전 만족 5 

487 배송도 빨리 오고 에어팟 진짜 이쁩니다!
하이마트에서 사길 잘햇다 생각합니다 잘 쓰겟습니다 5 

488 빠른배송에가성비좋음 5 

489 상품도 빨리 왔고 제품도 굿 5 

490 저렴한거 쓰려다 결국은 에어팟으로 오지싶어
한방에 왔는데 신세계네요~
배송빠르구 제품만족합니다^^ 5 

491 배송 엄청 빠르고 상품도 엄청 좋아여 10월 제조 상품 왓네용 ㅎㅎㅎㅎ 5 

492 뽑기도 잘 됐고 배송도 만족 상품도 만족입니다. 5 

493 너무 편안하고 좋습니다. 강력추천합니다. 5 

494 가격도 배송도 상품도 다 마음에 들어요 !! 에어팟 완전 신세계입니다 ^ ^ 적극 추천 합니다 5 

495 배송도 너무 빠르고 제품도 만족합니다. 아침에 주문했는데 오후에 발송해서 다음날 받았어요. 가격도 좋고 제품도 좋고 배송도 좋았네요. 제품도 좋아요. 무선 살까 고민을 많이 했는데 유선도 만족해요. 5 

496 배송이 좀 느렸지만 상품은 만족합니다.
케이스 색상을 고를수 있었더라면 더 좋았을거같아요. 5 

497 비교적 저렴한 가격에 잘 구매했습니다. 배송도 빠르고 공식 제품이라 맘에 들어요. 5 

498 배송진짜빨라요 대박 !!! 상품도 좋고 문제없어요 판매자님많이파세요 5 

499 좋은 제품 저렴하게구매 잘 하였습니다.
최고네요. 5 

500 배송빠르고 제품 만족합니다. 잘사용할께요. 

...

9 포장도 좋고 제품도 잘 있네요 근데 마감은 좀 아쉽네요 실리콘 케이스도 좀 더럽고 (끼우시기전에 안쪽에 검은색이 묻어잇으니 잘 확인하셔요) 가격대비 만족입니다 정품인증도 되니 뭐 됫죠+ (사용3일차 후기)뚜껑 유격잇음유닛마감은 쓰다보니 괜찮아짐제품특성상 슬슬 철가루가 생김배터리정상 페어링 양호 내부유닛 상태양호뚜껑유격땜시 틱틱 소리나는데 건 좀 빡치네요정가주고 살만한 제품은 아닌듯 십일절만 아니면 애플공홈 추천 3 

10 운전하느라 필요성을 못느끼다 그냥 충동구매했는데 신세계네요 ㅎㅎㅎ 컴팩트한게 너무 귀엽구 잘들려요! 다만 리뷰처럼 마감이 조금 깔끔하지 못해서.. 역시 마데인코리아가 좋은건가 싶기도 했지만 좋아요 ㅎㅎ 저는 케이스 바꾸는 맛에.... 5 

11 사진상 흐리게 보이는데 갈라짐 있었어요 만졌을 때 매끄럽지고 않고 조금이지만 갈라진 부분 튀어나와있었어요 교환문의 했는데 교환안된다네요그냥 찍히거나 긁히기만 했었어도 괜찮았을텐데...마감처리도 별로에요실망입니다 그냥 공홈에서 사세요 1 

12 절대 에어팟은 구매하지 마세요. 아니 여기 판매자에게서는 구매하지 마세요. 조립부분 마무리가 엉망이에요 저거 귀에 꽂으면 어떻게 될까요? 저런 제품팔고 반품은 안된데요. 두개 샀는데 하나는 깨끗하고 쓸만한데, 이건 누가봐도 완전 불량이잖아요. 근데 판매자가 대꾸가 없어요. 교환.환불 다 안된데요. 애플 갑질은 익히 알지만 직접 당하네요. 에어팟 사지마세요. 가격대비 완전 저질입니다. 살이 닫는 부분을 저렇게 만들다니 만들고 검수도 안하나 봐요. 사려는분들 이 리뷰보고 절대 구매하지 마세요. 1 

13 선물용으로 삿는데 개봉후 상태 점검해보니 에어팟 뚜껑 살짝 흔들림 유닛마감이 깔끔하지 않고 실리콘? 자국이 보임 하지만 여친이 그냥 쓰겟다고해서 씀 한달 정도 썻는데 에어팟 한쪽유닛안들림 새상품 파는건 맞나요?정품 등록은 잘되는데 새상품인지 긴가민가함 2 

14 마감이 들떠있다는 후기가 많아서 혹시 가품인가하고 걱정많이했어요 역시나 받아보니 마감이 좋지는

14 구매한지 1일만에 배송 완료 되었습니다.배송은 매우 빠르고, 아이폰에 테스트 하였고 아내가 너무 잘 들리고 깨끗한 음질이라고좋아라 합니다. 실물을 봤는데 역시 애플제품이라 그런지 마감도 확실히 다르고디자인도 아주 이쁘네요,많이 파세요. 5 

15 배송 이틀 만에 왔구여 정품 인증 바로 했구여 포장도 깔끔하게 왔어요 하자 하나없이 깔끔하고 좋아요 지인들이 무조건 공홈에서 사라고 해서 조금 불안하긴 했는데 받고나니까 그런거 1도 없고 싸게 잘산거 같다는 생각밖에 안드네요 감사함돠 잘쓸게요~~ 5 

16 에어팟 사용 후 삶의 질이 확 증가했습니다. 확실히 에어팟 사고 난 뒤에는 기존 이어폰 줄은 성가시게 느껴져요. 돈 좀 더 얹더라도 무선 충전이 편한 것 같아요. 그리고 작아서 잘 안 보이고 안 찾아지니 케이스는 쨍한 색의 톡톡 튀는 디자인 추천해용>.< 5 

17 출고는 빨랐으나 택배사에서 하루 지연되서 월요일 출고 수요일에 받았습니다. (한진 믿거)간혹 마감 불량이라든가 기기 결함이 있다고 교환한다는 후기가 보여 불안했습니다만 제껀 외형, 기기작동 모두 괜찮았네요. 솔직히 마감에 허술한 부분은 신경쓰이지 않았지만 기계 자체 결함은 안고 갈 수 없다 생각했거든요..결론적으론 유선 이어폰 사용하다 에어팟 사용하니 신세계입니다. 너무 편해요. 고민하는 분들은 질러도 후회없을 것 같습니당 5 

18 배송진짜 너무 맘에 들었어요 작은 박스에 제주도인데 2일걸려서 직접 수령했네요. 비싼제품이라 걱정했는데 오는시간까지 알려줘서 잘 받았어요. 타사 블루투스 이어폰 써봤는데 이건 정말 가볍고 착용감이 크지 않아서 좋더라구요 디자인도 좋구요. 양쪽 다 음질 좋고 기기와 연결도 쉽게 ？營윱求？. 많이파새요 5 

19 배송 진짜 빠르네요 시킨지 24시간도 안돼서 왔어요 ㅋㅋㅋ상품 깔끔하고 좋아요 열자마자 바로 페어링되네요!근데 케이스 뽑기 실패했네요..분명 후기엔 흰색 케이스왔다는것만 봤는데 전 왜 빨간색이죠ㅠㅠ돈아끼려다가 쓰레기만 생긴느낌 ㅋㅋㅋ새로 사려구요케이스 충

152 오전 주문해서 당일 발송, 바로 다음날에 받았습니다! 포장 및 사은품 만족스럽고, 다행히 뽑기운도 좋은지, 제품 외관이나 성능 (아직까진) 만족스럽습니다. 5 

153 마감 깔끔하고 좋은걸로 왔어요 만족해요 5 

154 음질도 좋고 디자인도 예뻐서 굿 5 

155 충천도 빠르고 음질도 좋고. 디자인도 이쁘네요.잘 사용하겠습니다. 사업 더욱더 번창하세요 5 

156 에어팟은 항상 사는거지만 최고네요 품질도 좋고 디자인도 예쁘고,, 이번에 프로 나왓지만 싸게 산다면 에어팟 2도 고려해볼만하다고 생각합니다 프로도 좋고요. 감사합니다 다음에도 또 구매할게요 5 

157 2주 간 사용해본결과 아주 만족스럽고 상태며 외관이며 다 맘에 들었습니다!^^ 5 

158 좋아요. 음질도 좋고 깔끔하니 좋네요~~다음에도 또 이용할게요^^ 5 

159 배송이 엄청빠르고 디자인도 너무 예쁘네요 5 

160 깔끔하고 이쁩니당! 5 

페이지 9 

161 배송도 하루만에 오고 제품도 너무 깔끔하니 좋아요!짱짱!! 5 

162 모양귀엽고, 음질좋고, 착용감 좋음. 단, 가격대가 비싼듯.. 5 

163 사용하기 편하고 디자인도 이뻐서 상품에 아저 만족합니다. 배송도 매우 빨라서 좋았습니다. 4 

164 디자인도 이쁘고 마음에 듭니다 음향도 좋은거 갔아요 4 

165 방금 열어서 케이스끼워봤는데 너무이쁘고 좋네요~~ 5 

166 좋아요 잘들리고 예쁘고..만족합니다 5 

167 외관이나 이어폰 모두 깔끔하네요 5 

168 감사합니다 이쁘고 좋아요 5 

169 이쁘고좋아요 삶의질향상 4 

170 이쁘고 좋아요 감사요 4 

171 배송도 빠른데다가! 포장도 깔끔해서 좋았어요! 정품인증도 잘 되서 기분이 좋네요! 감사히 잘 쓰겠습니다^--^!! 5 

172 딸한테 사줬는데 너무 좋아해요 예쁘고 만족합니다 5 

173 에어팟2세대 디쟌 품질 음질 최고예용~^^ 5 

174 제품 디자인이 평범하고 아주 좋습니다 4 

175 만족하고 깔끔해서 좋아요
응질도 

4 갤럭시 쓰는데 기존 LG 톤플러스 블투가 망가져서 무상보증기간도 지나고 ㅠㅠ 눈팅만 하던 2세대를 구매했어요. 호환에 대한 걱정은 이미 블로그를 통해 많이 봤고, 초기 세팅은 회사 동료분 아이폰에 등록해서 했네요~ 배터리는 거의 완충상태고 무선 충전도 잘대요~ 영업일해서 하루 종일 끼고 있는데 귀가 안아프네요. 플랜트로닉스것도 써봣는데 3시간 넘음 귀아파서 빼고 있었거든요. 음질은 거의 부드러운 중저음의 느낌? 목소리도 잘 들린다 하고, 총알배송에 주문한 담날 바루 사용중입니다. 넘 맘에 들어요. 악세사리 충동 불어서 케이스랑 키링도 구매하고 ㅎㅎ 만족스러워요~ 5 

5 배송진짜 너무 맘에 들었어요 작은 박스에 제주도인데 2일걸려서 직접 수령했네요. 비싼제품이라 걱정했는데 오는시간까지 알려줘서 잘 받았어요. 타사 블루투스 이어폰 써봤는데 이건 정말 가볍고 착용감이 크지 않아서 좋더라구요 디자인도 좋구요. 양쪽 다 음질 좋고 기기와 연결도 쉽게 ？營윱求？. 많이파새요 5 

6 유선 이어폰 쓰다가 이거 쓰니 완전 신세계네요. 핸폰은 밖에 두고 화장실에서 일 볼 때도 끊김없이 쓸 수 있어요. 이렇게 편할수가 ㅋㅋㅋ 사이즈도 귀에 쏙 들어가서 잘 빠지지도 않고 그렇다고 이물감 때문에 귀가 아프지도 않고. 좋은 가격에 득템. 많이 파십시오-! 5 

7 다른 블루투스 사용하고 있었지만 장시간 사용하면 귀가 아파서 고민하다가 에어팟 구입했습니다
배송도 빠르게 오고 제품도 이상 없이 잘되고 있습니다
다만 부재시 배송완료하고 따로 연락이 없는게 살짝 아쉽네요~ 감사합니다 5 

8 착용감도 좋고 음질도 좋아요 다만 음량 조절 시 1단계의 갭이 커서 많이 아쉬워요 음량 조절을 조금 세분화할 수 있다면 정말 좋겠어요 3 

9 동생이 잘사용하고있습니다. 근데 1시간 이상사용하면 귀가 아파요. 귀모양이 맞는 사람이 있을까요 ..? ㅠㅜ 애플에게 묻고싶다 4 

10 기존에 버즈를썼었는데 그건 계속끼고있으면귀가아프구 상대방이 잘안들린되서 통화품질이너무안좋았었는데 이거완전신세

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=83.0.4103.106)


In [ ]:
#에어팟 2세대
#패키지 import

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

name=['에어팟 1세대','에어팟 2세대']
category=['음질','품질','성능','기능','배터리 수명','사용성','조작성','디자인','착용감','휴대성']
#에어팟 2세대 검색 (유선)
ns_address_airpod="https://search.shopping.naver.com/detail/detail.nhn?nvMid=18622086330&query=%25EC%2597%2590%25EC%2596%25B4%25ED%258C%259F%25202%25EC%2584%25B8%25EB%258C%2580%2520%25EC%259C%25A0%25EC%2584%25A0%25EC%25B6%25A9%25EC%25A0%2584&NaPm=ct%3Dkbb1kefs%7Cci%3D145113ad84d3bb4c745b6faa37d4d9894a9d33fb%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dad9562496841c80a6f639b5837f07d5e88b01927"

shoppingmall_review="/html/body/div[2]/div[2]/div[2]/div[3]/div[1]/div/ul/li[3]/a/strong"
category1="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[3]/a/span" #음질
category2="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[6]/a/span" #품질
category3="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[7]/a/span" #성능
category4="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[8]/a/span" #기능
plus="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/a" #더보기
category5="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[17]/a/span"  #배터리 수명
category6="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[10]/a/span"  #사용성
category7="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[15]/a/span"  #조작성
#향상되지않은기능
category8="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[5]/a/span" #디자인
category9="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[11]/a/span" #착용감
category10="/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[1]/div[2]/div/ul/li[16]/a/span" #휴대성



header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/jmlee/Downloads/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address_airpod)
req = requests.get(ns_address_airpod)
html = req.text 
soup = BeautifulSoup(html, "html.parser")

#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df2=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df2.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df2.loc[n]=[name,category,'null','null']
        n+=1    
    return df2

def save():
    if not os.path.exists('output3.csv'):
        df2.to_csv('output3.csv', mode='w')
    else:
        df2.to_csv('output3.csv',mode='a',header=False)
        
        
#쇼핑몰 리뷰 보기
d.find_element_by_xpath(shoppingmall_review).click()
sleep(2)

        
#"음질" 관련 리뷰 가져오기
d.find_element_by_xpath(category1).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[0]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"품질" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category2).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[1]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()



#"성능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category3).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[2]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"기능" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category4).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[3]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"배터리 수명" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category5).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[4]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

#"사용성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category6).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[5]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category7).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[6]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


#"조작성" 관련 리뷰 가져오기
up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category8).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[7]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()

up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category9).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[8]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            


up="/html/body/div[2]/a" #맨위로 이동
d.find_element_by_xpath(up).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(plus).click() #스크롤 건드리면 안됨
d.find_element_by_xpath(category10).click() #스크롤 건드리면 안됨
name_=name[1]  #에어팟 2세대
category_=category[9]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[1]/a/span[2]/strong').text
            stars.append(star)
            review=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]').text
            reviews.append(review)
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
            
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a[12]').click()
            else : next_page=d.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
df2=add_dataframe(name_,category_,reviews,stars,cnt)
save()


In [19]:
dataset = pd.read_csv("output.csv", index_col=0)  #csv에서 가져오기
print(dataset)

       type category                                             review  star
1   에어팟 1세대       음질  2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...     5
2   에어팟 1세대       음질  배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...     5
3   에어팟 1세대       음질  아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...     5
4   에어팟 1세대       음질  다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...     4
5   에어팟 1세대       음질  택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...     5
..      ...      ...                                                ...   ...
9   에어팟 2세대      사용성  구하기 힘든 물건 판매해 주시고\n저렴한 판매와 빠른 배송 감사 드립니다.\n1세대...     5
1   에어팟 2세대      조작성  구입일이 아니라 제조일로부터 일정기간 경과된 제품이라 정품등록에 시간이 많이 소요되...     3
2   에어팟 2세대      조작성  칼배송!\n에어팟 처음 쓰는데 사용법도 간단합니다!\n정품 등록하는데 시리얼넘버는 ...     5
3   에어팟 2세대      조작성                                    품질 좋아요 간편해서 좋고.     5
4   에어팟 2세대      조작성  우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...     5

[1243 rows x 4 columns]
